In [ ]:
import sys
import glob
import os
import re
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import HuberRegressor
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import FuncFormatter, MaxNLocator
import numpy as np
from datetime import datetime
from typing import Dict, Optional, List
from google.colab import drive
drive.mount('/content/drive')

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.0f}'.format)

# load dataframes
gen_lookup = pd.read_csv("/content/drive/Shareddrives/market_analysis_v2/gen_lookup.csv")
listings = pd.read_csv("/content/drive/Shareddrives/market_analysis_v2/listings.csv")
notes = pd.read_csv("/content/drive/Shareddrives/market_analysis_v2/notes.csv")

Mounted at /content/drive


In [ ]:
clients=[
    {
        "client":"anita_c",
        "max_listing_price":13500,
        "max_odometer":160,
        "model_gens":[
            "3_2",
            "3_3",
            "civic_",
            "jazz_3",
            "i30_"
        ]
    },
    {
        "client":"magesh_t",
        "max_listing_price":13500,
        "max_odometer":160,
        "model_gens":[
            "3_3",
            "civic_",
            "i30_"
        ]
    }
]

# Scripts

In [ ]:
import re
import pandas as pd
from typing import Dict, Optional, List

# --- Carsales/General Scrapes (CS) Constants ---
YEAR_MIN, YEAR_MAX = 1980, 2035
ORDER: List[str] = ['href', 'year_make_model', 'trim', "listed_price", 'transmission', 'odometer', 'seller_type']

YEAR_RE  = r'\b(19[89]\d|20[0-3]\d)\b'
PRICE_RE = r'^(?:AU\$|\$)\s*[\d,]+(?:\.\d{2})?\b' # Made currency symbol mandatory
ODOM_RE  = r'^\s*\d+(?:,?\d{3})*K?\s*km\s*$' # Added optional 'K' for Facebook odometer format
URL_RE   = r'^(?:https?://|www\.)'
TX, SELLER = {'automatic', 'manual'}, {'private', 'dealer used'}

THRESH: Dict[str, float] = {
    'year_make_model': 0.50,
    "listed_price":           0.60,
    'transmission':    0.80,
    'odometer':        0.60,
    'seller_type':     0.70,
}

# --- Facebook Marketplace (FB) Constants ---
FB_ORDER: List[str] = ['href', 'year_make_model', 'listed_price', 'odometer', 'location']
THRESH_FB: Dict[str, float] = {
    'href':            0.80,
    'year_make_model': 0.50,
    'listed_price':    0.60,
    'odometer':        0.60,
    'location':        0.40,
}

# --- Predicates (Validation Rules) ---
def _ratio(mask: pd.Series) -> float:
    return float(mask.mean()) if len(mask) else 0.0

def _yr_ok(s: pd.Series) -> pd.Series:
    years = pd.to_numeric(s.astype(str).str.extract(YEAR_RE, expand=False), errors='coerce')
    return years.between(YEAR_MIN, YEAR_MAX)

PRED = {
    'year_make_model': lambda s: s.astype(str).pipe(_yr_ok) & s.astype(str).str.contains(r'[A-Za-z]', na=False),
    "listed_price":           lambda s: s.astype(str).str.match(PRICE_RE, na=False),
    'transmission':    lambda s: s.astype(str).str.strip().str.lower().isin(TX),
    'odometer':        lambda s: s.astype(str).str.match(ODOM_RE, flags=re.I, na=False),
    'seller_type':     lambda s: s.astype(str).str.strip().str.lower().isin(SELLER),
}

PRED_FB = {
    'year_make_model': lambda s: s.astype(str).pipe(_yr_ok) & s.astype(str).str.contains(r'[A-Za-z]', na=False),
    'listed_price':    lambda s: s.astype(str).str.match(PRICE_RE, na=False),
    'odometer':        lambda s: s.astype(str).str.match(ODOM_RE, flags=re.I, na=False),
}

# --- Core Identification Functions ---
def identify_columns(df: pd.DataFrame) -> Dict[str, Optional[str]]:
    """Identifies and maps raw DataFrame columns to canonical Carsales/General columns."""
    cols = list(df.columns)
    if not cols:
        return {k: None for k in ORDER}

    href_col = cols[0]

    # Exclude URL-like columns from other detection logic
    url_ratio = {c: _ratio(df[c].astype(str).str.contains(URL_RE, case=False, na=False)) for c in cols}
    urlish = {c for c, r in url_ratio.items() if r >= 0.50}
    blocked = {href_col} | urlish

    remaining = [c for c in cols if c not in blocked]
    picks = {t: None for t in PRED}

    for t in PRED:
        if not remaining:
            break
        scores = {c: _ratio(PRED[t](df[c])) for c in remaining}
        best_col, best_score = max(scores.items(), key=lambda kv: kv[1])
        if best_score >= THRESH[t]:
            picks[t] = best_col
            remaining.remove(best_col)

    trim_col = None
    ymm = picks.get('year_make_model')
    if ymm in cols:
        i = cols.index(ymm)
        if i + 1 < len(cols):
            trim_col = cols[i + 1]

    return {'href': href_col, **picks, 'trim': trim_col}

def identify_fb_columns(df: pd.DataFrame) -> Dict[str, Optional[str]]:
    """Identifies and maps raw DataFrame columns to canonical Facebook Marketplace columns.
    Note: 'href' is assumed to be the first column and is handled by clean_fb directly.
    """
    cols = list(df.columns)
    if not cols:
        return {k: None for k in FB_ORDER}

    picks = {t: None for t in FB_ORDER}
    remaining = set(cols)

    # 'href' is now handled externally by clean_fb and is assumed to be the first column
    # So we set it to None here or simply don't try to identify it.
    # We explicitly remove the first column from 'remaining' as it's the href
    if cols and cols[0] in remaining:
        remaining.remove(cols[0])
    picks['href'] = None # No longer identified by this function

    # Identify 'year_make_model', 'listed_price', 'odometer'
    for t in ['year_make_model', 'listed_price', 'odometer']:
        if not remaining:
            break
        scores = {c: _ratio(PRED_FB[t](df[c])) for c in remaining}
        if scores:
            best_col, score = max(scores.items(), key=lambda kv: kv[1])
            if score >= THRESH_FB[t]:
                picks[t] = best_col
                remaining.remove(best_col)

    # Assign 'location', often found in column 'c' or as the last remaining column
    if picks['location'] is None:
        if 'c' in remaining:
            picks['location'] = 'c'
            remaining.remove('c')
        elif len(remaining) == 1:
            picks['location'] = remaining.pop()

    return picks

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from typing import Dict, Optional, List

def clean_cs(df: pd.DataFrame, save_raw: bool = False) -> pd.DataFrame:
    """
    Business Logic for clean_cs function:

    This function processes raw DataFrame outputs from Carsales/General web scrapes to standardize
    and clean vehicle listing data into a consistent format for analysis.

    Key steps and business rules:
    1.  **Raw Data Preservation (Optional):** If `save_raw` is True, the original DataFrame
        is saved to a timestamped CSV, and a 'raw' column (filename) is added to the output.
    2.  **Column Identification:** Dynamically maps raw DataFrame columns to canonical names
        ('href', 'year_make_model', 'listed_price', 'odometer', etc.) using `identify_columns`.
    3.  **Data Extraction & Standardization:**
        *   Cleans 'href' by removing query parameters and `http(s)://www.` prefix.
        *   Splits 'year_make_model' into 'year', 'make', and 'model'; converts 'year' to integer.
        *   Converts 'listed_price' and 'odometer' to integer, removing non-numeric characters.
        *   Transforms 'odometer' values from 'km' to '000 km' (e.g., 180,000 km -> 180).
    4.  **Output Structure:** Returns a DataFrame with a standardized set of columns for consistency.
    """
    raw_col_value = None
    if save_raw:
        raw_data_dir = 'data/raws'
        os.makedirs(raw_data_dir, exist_ok=True)
        timestamp = datetime.now()
        raw_filename = ''
        while True:
            raw_filename = os.path.join(raw_data_dir, f"raw_carsales_data_{timestamp.strftime('%Y%m%d_%H%M%S')}.csv")
            if not os.path.exists(raw_filename):
                break
            timestamp += timedelta(seconds=1)
        df.to_csv(raw_filename, index=False)
        raw_col_value = os.path.basename(raw_filename)

    if 'identify_columns' not in globals():
        raise NameError("Function 'identify_columns' not found. Please ensure 'constants_and_helpers.py' or cell 'gECV1vdedUm0' has been executed.")

    out = pd.DataFrame()
    if not df.empty and len(df.columns) > 0:
        out['href'] = df.iloc[:, 0]

    mapping = identify_columns(df)
    for col in ['year_make_model', 'trim', "listed_price", 'transmission', 'odometer', 'seller_type']:
        src = mapping.get(col)
        if src is not None and src != out['href'].name:
            out[col] = df[src]

    if save_raw and raw_col_value:
        out['raw'] = raw_col_value

    if 'year_make_model' in out.columns:
        split_cols = out['year_make_model'].astype(str).str.split(expand=True, n=2)
        if 0 in split_cols.columns:
            out['year'] = pd.to_numeric(
                split_cols[0].astype(str).str.replace(r'[^\d]', '', regex=True),
                errors='coerce'
            ).astype('Int64')
        else:
            out['year'] = pd.NA
        out['make'] = split_cols[1] if 1 in split_cols.columns else pd.NA
        out['model'] = split_cols[2] if 2 in split_cols.columns else pd.NA
    else:
        out[['year', 'make', 'model']] = pd.NA

    if 'href' in out.columns:
        out['href'] = out['href'].astype(str).str.split('?').str[0] # Remove query parameters

    for col in ["listed_price", 'odometer']:
        if col in out.columns:
            out[col] = pd.to_numeric(
                out[col].astype(str).str.replace(r'[^\d]', '', regex=True),
                errors='coerce'
            ).astype('Int64')

    if 'odometer' in out.columns:
        out['odometer'] = out['odometer'] // 1000

    final_cols = ['href', 'year', 'make', 'model', "listed_price", 'trim', 'odometer', 'seller_type']
    if save_raw:
        final_cols.insert(0, 'raw')
    return out[[c for c in final_cols if c in out.columns]]

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from typing import Dict, Optional, List

def clean_fb(df: pd.DataFrame, save_raw: bool = False) -> pd.DataFrame:
    """
    Business Logic for clean_fb function:

    This function processes raw DataFrame outputs from Facebook Marketplace scrapes to standardize
    and clean vehicle listing data into a consistent format for analysis.

    Key steps and business rules:
    1.  **Raw Data Preservation (Optional):** If `save_raw` is True, the original DataFrame
        is saved to a timestamped CSV, and a 'raw' column (filename) is added to the output.
    2.  **Column Identification:** Dynamically maps raw DataFrame columns to canonical names
        ('href', 'year_make_model', 'listed_price', 'odometer', 'location') using `identify_fb_columns`.
    3.  **Data Extraction & Standardization:**
        *   Cleans 'href' by removing query parameters and `http(s)://www.` prefix.
        *   Splits 'year_make_model' into 'year', 'make', and 'model'; converts 'year' to integer.
        *   Converts 'listed_price' and 'odometer' to integer, removing non-numeric characters.
        *   Filters out listings with 'listed_price' explicitly marked as "free".
    4.  **Data Quality Filtering:** Drops rows with missing (`pd.NA`) values in critical columns
        ('listed_price', 'odometer', 'year') to ensure data integrity. Also removes listings
        with a placeholder 'listed_price' of 12345.
    5.  **Output Structure:** Returns a DataFrame with a standardized set of columns for consistency.
    """
    raw_col_value = None
    if save_raw:
        raw_data_dir = 'data/raws'
        os.makedirs(raw_data_dir, exist_ok=True)
        timestamp = datetime.now()
        raw_filename = ''
        while True:
            raw_filename = os.path.join(raw_data_dir, f"raw_facebook_data_{timestamp.strftime('%Y%m%d_%H%M%S')}.csv")
            if not os.path.exists(raw_filename):
                break
            timestamp += timedelta(seconds=1)
        df.to_csv(raw_filename, index=False)
        raw_col_value = os.path.basename(raw_filename)

    if 'identify_fb_columns' not in globals():
        raise NameError("Function 'identify_fb_columns' not found. Please ensure 'constants_and_helpers.py' or cell 'gECV1vdedUm0' has been executed.")

    out = pd.DataFrame()
    if not df.empty and len(df.columns) > 0:
        out['href'] = df.iloc[:, 0]

    mapping = identify_fb_columns(df)
    for canonical_col, src_col in mapping.items():
        if canonical_col != 'href' and src_col is not None and src_col in df.columns:
            out[canonical_col] = df[src_col]

    if save_raw and raw_col_value:
        out['raw'] = raw_col_value

    if 'year_make_model' in out.columns:
        split_df = out['year_make_model'].astype(str).str.split(expand=True, n=2)
        if 0 in split_df.columns:
            out['year'] = split_df[0].astype(str).str.replace(r'[^0-9]', '', regex=True).replace('', pd.NA).astype(float).astype('Int64')
        else:
            out['year'] = pd.NA
        out['make'] = split_df[1] if 1 in split_df.columns else pd.NA
        out['model'] = split_df[2] if 2 in split_df.columns else pd.NA
    else:
        out[['year', 'make', 'model']] = pd.NA

    if 'href' in out.columns:
        out['href'] = out['href'].astype(str).str.split('?').str[0] # Remove query parameters

    for col in ["listed_price", 'odometer']:
        if col in out.columns:
            if col == 'listed_price':
                out = out[out[col].astype(str).str.lower() != "free"]
            out[col] = pd.to_numeric(
                out[col].astype(str).str.replace(r'[^0-9]', '', regex=True),
                errors='coerce'
            ).astype('Int64')

    cols_to_check_for_na = []
    if 'listed_price' in out.columns: cols_to_check_for_na.append('listed_price')
    if 'odometer' in out.columns: cols_to_check_for_na.append('odometer')
    if 'year' in out.columns: cols_to_check_for_na.append('year')

    if cols_to_check_for_na:
        out = out.dropna(subset=cols_to_check_for_na)

    final_columns = ['href', 'year', 'make', 'model', "listed_price", 'odometer', 'location']
    if save_raw:
        final_columns.insert(0, 'raw')
    return out[[c for c in final_columns if c in out.columns]]

In [ ]:
import pandas as pd
from typing import Dict, Optional, List

def enrich_df(df: pd.DataFrame, gen_lookup: pd.DataFrame) -> pd.DataFrame:
    """Final clean after clean_cs or clean_fb, including generation assignment.

    Args:
        df (pd.DataFrame): The DataFrame to enrich.
        gen_lookup (pd.DataFrame): A lookup table for car generations.

    Returns:
        pd.DataFrame: The enriched DataFrame.
    """

    # --- 1. Add/Update date_scraped ---
    current_timestamp = pd.Timestamp.now().normalize()
    if 'date_scraped' in df.columns:
        df['date_scraped'] = df['date_scraped'].fillna(current_timestamp)
    else:
        df["date_scraped"] = current_timestamp

    # --- 2. Normalise make & model ---
    for col in ["make", "model"]:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.lower()
                .str.replace(r"[^a-z0-9]+", "", regex=True)
            )

    # --- Remove 'https://' or 'http://' and 'www.' from href ---
    if 'href' in df.columns:
        df['href'] = df['href'].astype(str).str.replace(r'^(https?://)?(www\.)?', '', regex=True)

    # --- 3. Ensure year is numeric ---
    if "year" in df.columns:
        df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")

    # --- 4. Calculate age ---
    if 'year' in df.columns:
        df['age'] = 2026 - df['year']

    # --- 5. Assign generation manually (no merge, no year_start/year_end contamination) ---
    df["gen"] = pd.NA

    for idx, row in gen_lookup.iterrows():
        mask = (
            (df["make"] == row["make"]) &
            (df["model"] == row["model"]) &
            (df["year"].between(row["year_start"], row["year_end"], inclusive="both"))
        )
        df.loc[mask, "gen"] = row["gen"]

    df["gen"] = df["gen"].astype("Int64")

    # --- 6. Create model_gen ---
    df["model_gen"] = df.apply(
        lambda r: f"{r['model']}_{r['gen']}" if pd.notna(r["gen"]) else None,
        axis=1
    )

    return df

In [ ]:
def remove_bad_listings(df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies filters to remove bad or undesirable listings from the DataFrame.
    This function is intended to be called after initial cleaning and data type conversions.

    Args:
        df (pd.DataFrame): The DataFrame to filter, expected to have 'year', 'listed_price', and 'odometer' columns.

    Returns:
        pd.DataFrame: The filtered DataFrame.
    """
    df_filtered = df.copy()

    # Price filters as specified by the user
    if 'listed_price' in df_filtered.columns:
        # Ensure listed_price is numeric for comparison
        df_filtered['listed_price'] = pd.to_numeric(df_filtered['listed_price'], errors='coerce')
        df_filtered = df_filtered[df_filtered["listed_price"] != 12345]
        df_filtered = df_filtered[df_filtered["listed_price"] > 3000]

    # Calculate age temporarily for the odometer filter if 'year' is available
    # Assuming 2026 is the reference year for age calculation based on other parts of the notebook
    if 'year' in df_filtered.columns:
        df_filtered['year'] = pd.to_numeric(df_filtered['year'], errors='coerce') # Ensure year is numeric
        temp_age = 2026 - df_filtered['year']
    else:
        temp_age = pd.Series(pd.NA, index=df_filtered.index) # Create a Series of NA for consistent operations

    # Odometer filter: odometer > 2 * age
    if 'odometer' in df_filtered.columns:
        # Ensure odometer is numeric
        df_filtered['odometer'] = pd.to_numeric(df_filtered['odometer'], errors='coerce')

        # Create a mask for rows where both odometer and temp_age are valid for comparison
        mask_valid_comparison = df_filtered['odometer'].notna() & temp_age.notna()

        # Filter out rows where (odometer is NOT > 2 * age) AND (the comparison is valid)
        # We keep rows where (odometer > 2 * age) OR (the comparison cannot be made due to NA values)
        df_filtered = df_filtered[~((df_filtered['odometer'] <= 2 * temp_age) & mask_valid_comparison)]

    return df_filtered

In [ ]:
def compare_new_listings(listings: pd.DataFrame, gen_lookup: pd.DataFrame):
    """
    Processes new listing files, cleans, enriches, and compares them against existing listings.

    Args:
        listings (pd.DataFrame): Existing DataFrame of car listings.
        gen_lookup (pd.DataFrame): Lookup table for car generations.

    Returns:
        Tuple[pd.DataFrame, int, int, int, int]: A tuple containing:
            - enriched_new_listings (pd.DataFrame): DataFrame of newly processed and enriched listings.
            - tot_new (int): Total count of truly new listings.
            - tot_updated (int): Total count of updated listings.
            - tot_unchanged (int): Total count of unchanged listings.
            - tot_tot (int): Total count of all listings processed from new files.
    """
    tot_new, tot_updated, tot_unchanged, tot_tot = 0, 0, 0, 0
    enriched_new_listings = pd.DataFrame()

    # Dynamically find new CSV files
    cs_files = glob.glob('/content/carsales*.csv')
    fb_files = glob.glob('/content/facebook*.csv')

    for file_path in cs_files + fb_files:
        df_raw = pd.read_csv(file_path)
        df_cleaned = None

        if 'carsales' in os.path.basename(file_path):
            df_cleaned = clean_cs(df_raw, save_raw=False)
        elif 'facebook' in os.path.basename(file_path):
            df_cleaned = clean_fb(df_raw, save_raw=False)
        else:
            print(f"Unknown file type: {file_path}")
            continue

        # Checking how many new, updated, unchanged listings
        df_comparison = pd.merge(
            df_cleaned,
            listings,
            on='href',
            how='left',
            suffixes=('_new', '_existing')
        )

        # Identify new listings
        new_listings_df = df_comparison[df_comparison['listed_price_existing'].isnull()]
        n_new = len(new_listings_df)

        # Identify matched listings
        matched_listings_df = df_comparison[df_comparison['listed_price_existing'].notnull()]

        # From matched_listings, identify updated listings
        updated_listings_df = matched_listings_df[
            matched_listings_df['listed_price_new'] != matched_listings_df['listed_price_existing']
        ]
        n_updated = len(updated_listings_df)

        # From matched_listings, identify unchanged listings
        unchanged_listings_df = matched_listings_df[
            matched_listings_df['listed_price_new'] == matched_listings_df['listed_price_existing']
        ]
        n_unchanged = len(unchanged_listings_df)

        # Calculate total listings for the current file
        n_total_listings = len(df_cleaned)

        # Print the comparison result for the current file
        print(f"{file_path}    \t {n_new=}   \t {n_updated=} \t {n_unchanged=} \t Tot {n_total_listings}")

        tot_new += n_new
        tot_updated += n_updated
        tot_unchanged += n_unchanged
        tot_tot += n_total_listings

        if df_cleaned is not None:
            df_enriched = enrich_df(df_cleaned, gen_lookup)
            enriched_new_listings = pd.concat([enriched_new_listings, df_enriched], ignore_index=True)


    print(f"\t \t \t \t {tot_new=} \t {tot_updated=}\t {tot_unchanged=} {tot_tot=}")

    # Check for missing values in enriched_new_listings after concatenation
    if not enriched_new_listings.empty:
        for col in ['model_gen', 'age', 'odometer']:
            if col in enriched_new_listings.columns and enriched_new_listings[col].isna().any():
                missing_count = enriched_new_listings[col].isna().sum()
                print(f"WARNING: Column '{col}' in enriched_new_listings has {missing_count} missing values.")


    return

In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
from typing import Dict, Optional, List
import glob # Import glob for file pattern matching

def integrate_listings(listings_df: pd.DataFrame, gen_lookup: pd.DataFrame) -> pd.DataFrame:
    """
    Integrates new car listings from '/content/carsales*.csv' and '/content/facebook*.csv' files into an existing listings DataFrame.

    Args:
        listings_df (pd.DataFrame): The existing DataFrame of car listings.
        gen_lookup (pd.DataFrame): The lookup table for car generations.

    Returns:
        pd.DataFrame: A new DataFrame (`listings_1`) with integrated, cleaned, and enriched listings,
                      with existing listings handled by keeping the most recent entry.
    """
    processed_dfs = []

    # Dynamically find new CSV files
    cs_files = glob.glob('/content/carsales*.csv')
    fb_files = glob.glob('/content/facebook*.csv')
    new_file_paths = cs_files + fb_files

    for file_path in new_file_paths:
        df_raw = pd.read_csv(file_path)
        df_cleaned = None

        if 'carsales' in os.path.basename(file_path):
            df_cleaned = clean_cs(df_raw, save_raw=False)
        elif 'facebook' in os.path.basename(file_path):
            df_cleaned = clean_fb(df_raw, save_raw=False)
        else:
            print(f"Unknown file type: {file_path}")
            continue

        if df_cleaned is not None:
            df_enriched = enrich_df(df_cleaned, gen_lookup)
            processed_dfs.append(df_enriched)

    if processed_dfs:
        new_listings_df = pd.concat(processed_dfs, ignore_index=True)

        # Define all possible columns that might exist in either DataFrame
        # Get columns from existing listings and new listings, handling potential differences
        all_cols = list(set(listings_df.columns) | set(new_listings_df.columns))

        # Reindex both DataFrames to ensure they have the same columns
        listings_aligned = listings_df.reindex(columns=all_cols, fill_value=pd.NA)
        new_listings_aligned = new_listings_df.reindex(columns=all_cols, fill_value=pd.NA)

        # Ensure 'date_scraped' is in datetime format for proper sorting
        listings_aligned['date_scraped'] = pd.to_datetime(listings_aligned['date_scraped'], errors='coerce')
        new_listings_aligned['date_scraped'] = pd.to_datetime(new_listings_aligned['date_scraped'], errors='coerce')

        # Explicitly cast dtypes of new_listings_aligned to match listings_aligned for common columns
        # This helps prevent FutureWarning and ensures consistent types across the concatenated DataFrame
        for col in all_cols:
            if col in listings_aligned.columns and col in new_listings_aligned.columns:
                if listings_aligned[col].dtype != new_listings_aligned[col].dtype:
                    try:
                        if pd.api.types.is_numeric_dtype(listings_aligned[col]):
                            if str(listings_aligned[col].dtype) == 'Int64':
                                new_listings_aligned[col] = new_listings_aligned[col].astype('Int64')
                            else:
                                new_listings_aligned[col] = pd.to_numeric(new_listings_aligned[col], errors='coerce').astype(listings_aligned[col].dtype)
                        else:
                            new_listings_aligned[col] = new_listings_aligned[col].astype(listings_aligned[col].dtype)
                    except (TypeError, ValueError):
                        pass # Keep original dtype if casting causes error

        # Concatenate the aligned Dataframes
        listings_1 = pd.concat([listings_aligned, new_listings_aligned], ignore_index=True)
    else:
        listings_1 = listings_df.copy()

    # Sort by href, then listed_price (lowest first), then date_scraped (most recent first), then drop duplicates keeping the first
    listings_1 = listings_1.sort_values(by=['href', 'listed_price', 'date_scraped'], ascending=[True, True, False])
    listings_1 = listings_1.drop_duplicates(subset=['href'], keep='first')
    listings_1 = remove_bad_listings(listings_1)

    # Ensure 'gen' column is Int64 after all operations
    listings_1['gen'] = listings_1['gen'].astype('Int64')

    print(f"Final listings_1 DataFrame has {len(listings_1)} unique listings after merging and de-duplication.")
    return listings_1

In [81]:
import pandas as pd
from datetime import date
from typing import Optional, List, Dict, Union

def get_best_listings(listings_lr: pd.DataFrame, client: Optional[str] = None, start_date: Optional[date] = None) -> pd.DataFrame:
    """
    Filters car listings based on universal standards, client-specific criteria, and date.
    Returns a single DataFrame with unique listings (de-duplicated by href).

    Args:
        listings_lr (pd.DataFrame): The DataFrame of car listings with regression results.
        client (Optional[str]): The name of a specific client to filter for. If None, considers all clients.
        start_date (Optional[date]): A date. If provided, keeps listings scraped on or after this date.
                                   If None, keeps only the most recent listings scraped.

    Returns:
        pd.DataFrame: A de-duplicated DataFrame of eligible listings across all clients,
                      sorted by excess_value descending, including client eligibility flags.
    """

    # 1. Apply Universal Filters
    listings_filtered = listings_lr[
        (listings_lr['odometer'] > 4 * listings_lr['age']) &
        (listings_lr['listed_price'] < 0.95 * listings_lr['market_value'])
    ].copy()

    if listings_filtered.empty:
        print("No listings remain after universal filters.")
        return pd.DataFrame()

    # 2. Apply Date Filter (start_date or most recent)
    if start_date:
        # Convert start_date to pandas Timestamp for comparison
        start_timestamp = pd.Timestamp(start_date).normalize()
        listings_filtered = listings_filtered[listings_filtered['date_scraped'] >= start_timestamp]
    else:
        # Identify the most recent date_scraped across all listings
        most_recent_date = listings_filtered['date_scraped'].max()
        # Filter to keep all listings on the same day as the most recent date
        listings_filtered = listings_filtered[listings_filtered['date_scraped'].dt.date == most_recent_date.date()]

    if listings_filtered.empty:
        print("No listings remain after date filtering.")
        return pd.DataFrame()

    # 3. Evaluate Client Suitability and add flags
    global clients # Access the global clients list

    for client_info in clients:
        current_client_name = client_info['client']
        max_price = client_info['max_listing_price']
        max_odometer = client_info['max_odometer']
        model_gens_allowed = client_info['model_gens']

        # Initialize client eligibility column
        listings_filtered[f'client_{current_client_name}'] = False

        # Price and odometer conditions
        price_cond = listings_filtered['listed_price'] <= max_price
        odometer_cond = listings_filtered['odometer'] <= max_odometer

        # Model generation condition
        model_gen_cond = pd.Series(False, index=listings_filtered.index)
        if 'model_gen' in listings_filtered.columns:
            for allowed_gen_pattern in model_gens_allowed:
                model_gen_cond = model_gen_cond | listings_filtered['model_gen'].astype(str).str.startswith(allowed_gen_pattern)

        # Combine all conditions for the current client
        listings_filtered.loc[price_cond & odometer_cond & model_gen_cond, f'client_{current_client_name}'] = True

    # 4. Collect top 10 for each client, combine, and de-duplicate
    all_client_top_listings_list = []
    if 'excess_value' not in listings_filtered.columns:
        print("Error: 'excess_value' column is missing for sorting.")
        return pd.DataFrame()

    for client_info in clients:
        current_client_name = client_info['client']
        client_eligibility_col = f'client_{current_client_name}'

        # Filter for listings eligible for the current client
        eligible_for_current_client = listings_filtered[
            listings_filtered[client_eligibility_col]
        ].copy()

        if not eligible_for_current_client.empty:
            # Sort by excess_value and take top 10 for THIS client
            top_10 = eligible_for_current_client.sort_values(by='excess_value', ascending=False).head(10)
            all_client_top_listings_list.append(top_10)

    if all_client_top_listings_list:
        # Concatenate all top 10 lists
        union_df = pd.concat(all_client_top_listings_list, ignore_index=True)

        # De-duplicate based on 'href', keeping the one with the highest 'excess_value'
        # This ensures a true 'union' without duplicate listings
        union_df = union_df.sort_values(by='excess_value', ascending=False).drop_duplicates(subset=['href'], keep='first')

        # Define desired column order, ensuring client eligibility columns are included
        final_cols = [
            'href', 'model_gen', 'year', 'odometer', 'seller', 'date_scraped',
            'market_value', 'listed_price', 'excess_value'
        ] + [f'client_{c["client"]}' for c in clients]

        # Ensure all final_cols exist in union_df before selection
        final_cols = [col for col in final_cols if col in union_df.columns]

        return union_df[final_cols]
    else:
        print("No eligible listings found across all clients after selecting top 10s.")
        return pd.DataFrame()


In [70]:
import pandas as pd
import yaml
import os
from datetime import datetime, date
import numpy as np
from google.colab import files # Import files for download functionality

def output_shortlist(shortlist: pd.DataFrame, notes_df: pd.DataFrame):
    """
    Consolidates listing data from shortlist and notes DataFrames and saves it to a YAML file.
    Dynamically adds client eligibility flags from shortlist columns.

    Args:
        shortlist (pd.DataFrame): DataFrame containing shortlisted listings.
        notes_df (pd.DataFrame): DataFrame containing notes associated with listings.
    """

    # Helper function to convert pandas-specific types to standard Python equivalents
    def to_python_type(value):
        if pd.isna(value):
            return None
        if isinstance(value, pd.Timestamp):
            return value.to_pydatetime() # Convert pandas Timestamp to datetime object
        if isinstance(value, (pd.Int64Dtype, np.int64)):
            return int(value)
        if isinstance(value, (pd.Float64Dtype, np.float64)):
            return float(value)
        if isinstance(value, (date, datetime)): # Use datetime.date and datetime
            return value
        return value

    # Prepare notes_df
    prepared_notes = notes_df.copy()
    prepared_notes['timestamp'] = pd.to_datetime(prepared_notes['timestamp'], errors='coerce')
    prepared_notes.dropna(subset=['timestamp'], inplace=True)

    all_listings_data = []

    for idx, row in shortlist.iterrows():
        href = row['href']
        current_status = None
        current_notes = []

        matching_notes = prepared_notes[prepared_notes['href'] == href]

        if not matching_notes.empty:
            matching_notes_sorted = matching_notes.sort_values(by='timestamp', ascending=False)
            current_status = to_python_type(matching_notes_sorted.iloc[0]['status'])
            current_notes = [to_python_type(n) for n in matching_notes_sorted['note'].tolist() if pd.notna(n)]

        # Create a dictionary named listing_data with the specified order and format
        listing_data = {
            'title': f"{to_python_type(row['year'])}, {to_python_type(row['model_gen'])}, {int(to_python_type(row['odometer']))}k",
            'seller': to_python_type(row['seller']),
            'listed_price': to_python_type(row['listed_price']),
            'excess_value': int(to_python_type(row['excess_value'])), # Convert to int here
            'href': to_python_type(row['href']),
            'date_scraped': to_python_type(row['date_scraped']),
        }

        # Dynamically add client eligibility flags
        for col in shortlist.columns:
            if col.startswith('client_'):
                listing_data[col] = to_python_type(row[col])

        # Add status and notes
        listing_data['status'] = current_status
        listing_data['notes'] = current_notes

        all_listings_data.append(listing_data)

    output_filename = 'shortlist.yaml'
    # Write to a string buffer and then to file for download
    yaml_content = []
    for listing in all_listings_data:
        yaml_content.append('---\n') # Add separator before each listing
        yaml_content.append(yaml.dump(listing, allow_unicode=True, default_flow_style=False, sort_keys=False))
        yaml_content.append('\n') # Add an extra newline after each dumped listing for readability

    with open(output_filename, 'w') as file:
        file.writelines(yaml_content)

    files.download(output_filename)
    print(f"The YAML file '{output_filename}' has been generated and prompted for download with {len(all_listings_data)} listings.")


# Linear Regression

In [ ]:
def apply_regression(df: pd.DataFrame) -> (pd.DataFrame, pd.Series):
    """
    Applies Huber regression to the input DataFrame to predict car prices.

    Args:
        df (pd.DataFrame): The input DataFrame containing car listings.

    Returns:
        (pd.DataFrame, pd.Series): A tuple containing:
            - The DataFrame with 'market_value' and 'excess_value' columns added.
            - A Series of unscaled regression coefficients.
    """
    listings_lr = df.copy()

    # 1) Coerce numeric types
    listings_lr['year'] = pd.to_numeric(listings_lr['year'], errors='coerce')
    listings_lr['odometer'] = pd.to_numeric(listings_lr['odometer'], errors='coerce')
    listings_lr["listed_price"] = pd.to_numeric(listings_lr["listed_price"], errors='coerce')

    # 2) One-hot encode model_gen
    listings_lr["model_gen"] = listings_lr["model_gen"].astype(str)
    dummies = pd.get_dummies(listings_lr["model_gen"], prefix="mg_", prefix_sep="")

    # remove base category "civic_9" if it exists
    base_col = "civic_9"
    if base_col in dummies.columns:
        dummies = dummies.drop(columns=[base_col])

    listings_lr = pd.concat([listings_lr, dummies], axis=1)

    # 3) Build X, y & keep mask
    predictor_cols = ['age', 'odometer'] + list(dummies.columns)
    X = listings_lr[predictor_cols].astype(float)
    y = listings_lr["listed_price"].astype(float)

    keep = X.notna().all(axis=1) & y.notna()

    X_keep = X.loc[keep]
    y_keep = y.loc[keep]

    # 4) Scale predictors
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_keep)

    # 5) Fit Huber Regression
    huber = HuberRegressor(max_iter=1000, epsilon=1.5)
    huber.fit(X_scaled, y_keep)

    # 6) Predict & store results
    pred = huber.predict(X_scaled)
    listings_lr.loc[keep, "market_value"] = pred
    listings_lr.loc[keep, "excess_value"] = pred - listings_lr.loc[keep, "listed_price"]

    # 7) Recover coefficients on the original (unscaled) feature scale
    coef_scaled = huber.coef_
    mu = scaler.mean_
    sigma = scaler.scale_

    original_intercept = huber.intercept_ - np.sum(coef_scaled * (mu / sigma))
    original_coefs = coef_scaled / sigma

    coef_unscaled = pd.Series(
        np.concatenate([[original_intercept], original_coefs]),
        index=["intercept"] + predictor_cols
    )

    listings_lr = listings_lr.loc[:, ~listings_lr.columns.str.startswith("mg_")]

    return listings_lr, coef_unscaled

# Working

In [72]:
compare_new_listings(listings, gen_lookup)

/content/carsales (3).csv    	 n_new=8   	 n_updated=0 	 n_unchanged=0 	 Tot 8
/content/carsales (4).csv    	 n_new=11   	 n_updated=0 	 n_unchanged=0 	 Tot 11
/content/carsales (1).csv    	 n_new=14   	 n_updated=0 	 n_unchanged=0 	 Tot 14
/content/carsales (2).csv    	 n_new=21   	 n_updated=0 	 n_unchanged=0 	 Tot 21
/content/carsales.csv    	 n_new=8   	 n_updated=0 	 n_unchanged=0 	 Tot 8
/content/carsales (6).csv    	 n_new=15   	 n_updated=0 	 n_unchanged=0 	 Tot 15
/content/carsales (5).csv    	 n_new=22   	 n_updated=0 	 n_unchanged=0 	 Tot 22
/content/facebook (1).csv    	 n_new=25   	 n_updated=0 	 n_unchanged=0 	 Tot 25
/content/facebook (3).csv    	 n_new=29   	 n_updated=0 	 n_unchanged=0 	 Tot 29
/content/facebook.csv    	 n_new=32   	 n_updated=0 	 n_unchanged=0 	 Tot 32
/content/facebook (2).csv    	 n_new=53   	 n_updated=0 	 n_unchanged=0 	 Tot 53
	 	 	 	 tot_new=238 	 tot_updated=0	 tot_unchanged=0 tot_tot=238


In [73]:
# Add new listings to listings dataframe

# Call the function to integrate the listings
listings1 = integrate_listings(listings, gen_lookup)

print("\nFirst 5 rows of the newly created listings_1:")
display(listings1.head())

Final listings_1 DataFrame has 1097 unique listings after merging and de-duplication.

First 5 rows of the newly created listings_1:


,age,location,seller_type,date_scraped,seller,model_gen,year,make,odometer,href,model,gen,trim,listed_price
950,22,NaN,Dealer used,2025-12-07,<NA>,jazz_1,2004,honda,183,carsales.com.au/cars/details/2004-honda-jazz-vti-auto/OAG-AD-25139851/,jazz,1,VTi Auto F,4999
959,22,NaN,Dealer used,2025-12-07,<NA>,jazz_1,2004,honda,173,carsales.com.au/cars/details/2004-honda-jazz-vti-s-auto-my05/OAG-AD-25221211/,jazz,1,VTi-S Auto F MY05,6995
945,21,NaN,Private,2025-12-07,<NA>,jazz_1,2005,honda,270,carsales.com.au/cars/details/2005-honda-jazz-gli-auto-my05/SSE-AD-18911096/,jazz,1,GLi Auto F MY05,5900
967,21,NaN,Private,2025-12-07,<NA>,jazz_1,2005,honda,121,carsales.com.au/cars/details/2005-honda-jazz-vti-auto-my05/SSE-AD-19218247/,jazz,1,VTi Auto F MY05,7000
369,20,NaN,Dealer used,2025-12-07,<NA>,accordeuro_1,2006,honda,135,carsales.com.au/cars/details/2006-honda-accord-euro-luxury-auto-my06/OAG-AD-25285163/,accordeuro,1,Luxury Auto F MY06,8990


In [74]:
listings_lr, coefficients = apply_regression(listings1)

In [75]:
listings_lr.head()

,age,location,seller_type,date_scraped,seller,model_gen,year,make,odometer,href,model,gen,trim,listed_price,market_value,excess_value
950,22,NaN,Dealer used,2025-12-07,<NA>,jazz_1,2004,honda,183,carsales.com.au/cars/details/2004-honda-jazz-vti-auto/OAG-AD-25139851/,jazz,1,VTi Auto F,4999,4580,-419
959,22,NaN,Dealer used,2025-12-07,<NA>,jazz_1,2004,honda,173,carsales.com.au/cars/details/2004-honda-jazz-vti-s-auto-my05/OAG-AD-25221211/,jazz,1,VTi-S Auto F MY05,6995,4915,-2080
945,21,NaN,Private,2025-12-07,<NA>,jazz_1,2005,honda,270,carsales.com.au/cars/details/2005-honda-jazz-gli-auto-my05/SSE-AD-18911096/,jazz,1,GLi Auto F MY05,5900,2286,-3614
967,21,NaN,Private,2025-12-07,<NA>,jazz_1,2005,honda,121,carsales.com.au/cars/details/2005-honda-jazz-vti-auto-my05/SSE-AD-19218247/,jazz,1,VTi Auto F MY05,7000,7280,280
369,20,NaN,Dealer used,2025-12-07,<NA>,accordeuro_1,2006,honda,135,carsales.com.au/cars/details/2006-honda-accord-euro-luxury-auto-my06/OAG-AD-25285163/,accordeuro,1,Luxury Auto F MY06,8990,10142,1152


In [76]:
listings_lr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1097 entries, 950 to 685
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   age           1097 non-null   Int64         
 1   location      367 non-null    object        
 2   seller_type   404 non-null    object        
 3   date_scraped  1097 non-null   datetime64[ns]
 4   seller        0 non-null      string        
 5   model_gen     1097 non-null   object        
 6   year          1097 non-null   Int64         
 7   make          1097 non-null   object        
 8   odometer      1097 non-null   Int64         
 9   href          1097 non-null   object        
 10  model         1097 non-null   object        
 11  gen           1094 non-null   Int64         
 12  trim          436 non-null    object        
 13  listed_price  1097 non-null   Int64         
 14  market_value  1097 non-null   float64       
 15  excess_value  1097 non-null   Float64     

In [77]:
listings_lr.value_counts("date_scraped")

,count
date_scraped,
2025-12-07,1097


In [89]:
shortlist = get_best_listings(listings_lr)
shortlist

In [84]:
# Call the updated output_shortlist function
output_shortlist(shortlist, notes)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The YAML file 'shortlist.yaml' has been generated and prompted for download with 15 listings.
